In [122]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!which python
print(tf.__version__)

/home/lol/Desktop/Labs/AI/.venv/bin/python
2.2.0


In [123]:
import glob
import imageio

dataset = []

for path in glob.glob("./photos/1/crop/2/*.png"):
    img = imageio.imread(path)
    dataset.append(1^img//255)

images = np.array(dataset)

In [193]:
model = keras.Sequential([
    keras.layers.Input(shape=(32,32),name="input"),
    keras.layers.Flatten(input_shape=(32,32), name="flat"),
    keras.layers.Dense(units=32*32, activation='relu', name="process", input_dim=32*32),
    keras.layers.Dense(units=3, activation='softmax', name="categorize")
],name="categorizer")

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "categorizer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flat (Flatten)               (None, 1024)              0         
_________________________________________________________________
process (Dense)              (None, 1024)              1049600   
_________________________________________________________________
categorize (Dense)           (None, 3)                 3075      
Total params: 1,052,675
Trainable params: 1,052,675
Non-trainable params: 0
_________________________________________________________________


In [194]:
k = keras.utils.to_categorical(
    np.concatenate([
    np.array([0]*9),
    np.array([1]*9),
    np.array([2]*9)
]),
    num_classes=3
)

model.fit(images, k, epochs=6)


Epoch 1/6
1/1 [==============================] - 0s 2ms/step - loss: 1.1657 - accuracy: 0.2963
Epoch 2/6
1/1 [==============================] - 0s 939us/step - loss: 0.6266 - accuracy: 0.7407
Epoch 3/6
1/1 [==============================] - 0s 1ms/step - loss: 0.2774 - accuracy: 1.0000
Epoch 4/6
1/1 [==============================] - 0s 2ms/step - loss: 0.1566 - accuracy: 1.0000
Epoch 5/6
1/1 [==============================] - 0s 2ms/step - loss: 0.0953 - accuracy: 1.0000
Epoch 6/6
1/1 [==============================] - 0s 1ms/step - loss: 0.0546 - accuracy: 1.0000


In [195]:
model.predict(np.array([images[20]]))


array([[0.0118105 , 0.07769623, 0.9104933 ]], dtype=float32)

In [ ]:
plt.